In [1]:
import anatomist.api as anatomist

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from soma import aims
import json
from soma.qt_gui.qt_backend import Qt
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np

In [3]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module anacontrolmenu
loading module foldsplit
loading module simple_controls
loading module histogram
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

json_regions = "/neurospin/dico/data/deep_folding/history/2023-09_joel-flanker/Input/sulci_regions_gridsearch.json"

result_file = "/neurospin/dico/data/deep_folding/history/2023-09_joel-flanker/Output/flanker_per_region_on_HCP_ensemble.ods" 

In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [7]:
res = pd.read_excel(result_file)
# res = res[["Region (sulci_region_gridsearch)", "AUC", "std_auc"]].rename(columns = {"Region (sulci_region_gridsearch)": "region"})
res = res[~(res["region"] == "fronto-parietal_medial_face.")]
res = res[~(res["region"] == "S.C.-S.Pe.C.")]
res = res[~(res["region"] == "S.C.-S.Po.C.")]
res.head()

,Unnamed: 0,region,region_simple_name,Lobe,AUC,std_auc,AUC-1,std_auc-1,AUC-2,std_auc-2,...,std_auc_best_model,log_lr,std_log_lr,logp_AUC_criterion,logp_AUC_old_criterion,logp_AUC_best_val,logp_AUC_best_model,AUC_ensemble,std_auc_ensemble,logp_AUC_ensemble
0,15,S.F.marginal-S.F.inf.ant.,SFmarginal,frontal,0.609,0.009,0.548,0.054,0.609,0.009,...,0.061147,2.593849,0.494544,5.250836,1.223397,5.250836,1.045514,0.616450,0.002051,7.938063
1,5,S.C.-sylv.,central,frontal,0.578,0.026,0.578,0.026,0.585,0.015,...,0.011038,2.869598,0.500598,2.855748,2.855748,3.976741,4.386128,0.610390,0.014172,4.496263
2,0,CINGULATE.,cingulate,frontal,0.559,0.019,0.560,0.019,0.546,0.028,...,0.039295,2.653998,0.434632,2.970499,2.970499,2.005764,1.895694,0.571717,0.009008,4.533905
3,16,S.F.int.-S.R.,SFinternal,frontal,0.565,0.007,0.565,0.007,0.488,0.051,...,0.031525,3.004289,0.569990,4.901559,4.901559,0.159768,1.756349,0.571429,0.004875,5.587088
5,13,S.F.median-S.F.pol.tr.-S.F.sup.,SFmedian,frontal,0.558,0.009,0.554,0.008,0.558,0.009,...,0.026354,2.586598,0.405700,4.222804,4.178358,4.222804,2.662346,0.539105,0.007121,3.898488


In [8]:
def compute_logp(auc, std_auc):
    p = scipy.stats.ttest_ind_from_stats(mean1=auc, 
                              std1=std_auc, 
                              nobs1=5, 
                              mean2=0.5, 
                              std2=0, 
                              nobs2=2, 
                              alternative='greater',
                              equal_var=False)[1]
    return -np.log10(p)

In [9]:
# res["-log10_p"] =  res.apply(lambda x: compute_logp(x.AUC, x.std_auc), axis=1)
# res

In [10]:
def get_sulci(region):
    list_sulci = list(regions['brain'][f"{region}_right"].keys())
    list_sulci = ["S.F.int._right" if x=="paracingular._right" else x for x in list_sulci]
    return list_sulci

In [11]:
get_sulci("CINGULATE.")

['S.F.int._right', 'F.C.M.ant._right']

In [12]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,Unnamed: 0,region,region_simple_name,Lobe,AUC,std_auc,AUC-1,std_auc-1,AUC-2,std_auc-2,...,log_lr,std_log_lr,logp_AUC_criterion,logp_AUC_old_criterion,logp_AUC_best_val,logp_AUC_best_model,AUC_ensemble,std_auc_ensemble,logp_AUC_ensemble,sulcus
0,15,S.F.marginal-S.F.inf.ant.,SFmarginal,frontal,0.609,0.009,0.548,0.054,0.609,0.009,...,2.593849,0.494544,5.250836,1.223397,5.250836,1.045514,0.616450,0.002051,7.938063,"[S.F.marginal._right, S.F.orbitaire._right, S...."
1,5,S.C.-sylv.,central,frontal,0.578,0.026,0.578,0.026,0.585,0.015,...,2.869598,0.500598,2.855748,2.855748,3.976741,4.386128,0.610390,0.014172,4.496263,"[S.C._right, S.C.sylvian._right]"
2,0,CINGULATE.,cingulate,frontal,0.559,0.019,0.560,0.019,0.546,0.028,...,2.653998,0.434632,2.970499,2.970499,2.005764,1.895694,0.571717,0.009008,4.533905,"[S.F.int._right, F.C.M.ant._right]"
3,16,S.F.int.-S.R.,SFinternal,frontal,0.565,0.007,0.565,0.007,0.488,0.051,...,3.004289,0.569990,4.901559,4.901559,0.159768,1.756349,0.571429,0.004875,5.587088,"[S.F.int._right, S.R.inf._right, S.R.sup._right]"
5,13,S.F.median-S.F.pol.tr.-S.F.sup.,SFmedian,frontal,0.558,0.009,0.554,0.008,0.558,0.009,...,2.586598,0.405700,4.222804,4.178358,4.222804,2.662346,0.539105,0.007121,3.898488,"[S.F.median._right, S.F.polaire.tr._right, S.F..."


In [13]:
res = res.sort_values(by="AUC_ensemble", ascending=True)
res = res.explode("sulcus")
res.head()

,Unnamed: 0,region,region_simple_name,Lobe,AUC,std_auc,AUC-1,std_auc-1,AUC-2,std_auc-2,...,log_lr,std_log_lr,logp_AUC_criterion,logp_AUC_old_criterion,logp_AUC_best_val,logp_AUC_best_model,AUC_ensemble,std_auc_ensemble,logp_AUC_ensemble,sulcus
10,10,S.Or.,SOr,frontal,0.486,0.060,0.486,0.060,0.443,0.050,...,2.696498,0.241280,0.163095,0.163095,0.019173,0.013899,0.415296,0.044664,0.002888,S.Or._right
16,4,S.T.s.br.,STs_br,temporal,0.468,0.009,0.468,0.009,0.468,0.009,...,3.000318,0.329289,0.000326,0.000326,0.000326,0.065055,0.460895,0.011554,0.000355,F.I.P.r.int.1_right
16,4,S.T.s.br.,STs_br,temporal,0.468,0.009,0.468,0.009,0.468,0.009,...,3.000318,0.329289,0.000326,0.000326,0.000326,0.065055,0.460895,0.011554,0.000355,F.I.P.r.int.2_right
16,4,S.T.s.br.,STs_br,temporal,0.468,0.009,0.468,0.009,0.468,0.009,...,3.000318,0.329289,0.000326,0.000326,0.000326,0.065055,0.460895,0.011554,0.000355,S.T.s.ter.asc.ant._right
16,4,S.T.s.br.,STs_br,temporal,0.468,0.009,0.468,0.009,0.468,0.009,...,3.000318,0.329289,0.000326,0.000326,0.000326,0.065055,0.460895,0.011554,0.000355,S.T.s.ter.asc.post._right


In [14]:
for _, row in res.iterrows():
    print(row.sulcus)

S.Or._right
F.I.P.r.int.1_right
F.I.P.r.int.2_right
S.T.s.ter.asc.ant._right
S.T.s.ter.asc.post._right
S.GSM._right
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
F.I.P._right
F.I.P.Po.C.inf._right
S.Pe.C.inf._right
S.Pe.C.inter._right
S.Pe.C.marginal._right
S.Pe.C.sup._right
S.Pe.C.median._right
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
S.T.i.ant._right
S.T.i.post._right
S.T.s._right
S.T.pol._right
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
S.F.inter._right
S.F.sup._right
S.Or._right
S.Olf._right
S.Po.C.sup._right
F.I.P.Po.C.inf._right
F.C.L.r.retroC.tr._right
S.T.s._right
S.s.P._right
S.Pa.int._right
S.F.median._right
S.F.polaire.tr._right
S.F.sup._right
OCCIPITAL_right
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
F.P.O._right
S.

In [15]:
res[res.sulcus=="S.F.orbitaire._right"]

,Unnamed: 0,region,region_simple_name,Lobe,AUC,std_auc,AUC-1,std_auc-1,AUC-2,std_auc-2,...,log_lr,std_log_lr,logp_AUC_criterion,logp_AUC_old_criterion,logp_AUC_best_val,logp_AUC_best_model,AUC_ensemble,std_auc_ensemble,logp_AUC_ensemble,sulcus
0,15,S.F.marginal-S.F.inf.ant.,SFmarginal,frontal,0.609,0.009,0.548,0.054,0.609,0.009,...,2.593849,0.494544,5.250836,1.223397,5.250836,1.045514,0.61645,0.002051,7.938063,S.F.orbitaire._right


In [17]:
g = aims.read(Rspam_model)
ag = a.toAObject(g)



for vertex in g.vertices():
    vertex['AUC_ensemble'] = 0.

for _, row in res.iterrows():
    for vertex in g.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['AUC_ensemble']):
                vertex['AUC_ensemble'] = row.AUC_ensemble #-row.std_auc_ensemble

ag.setColorMode(ag.PropertyMap)
ag.setColorProperty('AUC_ensemble')
ag.notifyObservers()

w = a.createWindow('3D')
w.addObjects(ag)


In [ ]:
res = res.sort_values(by="-log10_p", ascending=True)

In [ ]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['-log10p'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['-log10_p']):
                vertex['-log10p'] = row['-log10_p']

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('-log10p')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)
